<a href="https://colab.research.google.com/github/danielstaley/Data-Pipelining/blob/master/Price_Competitiveness_Build_Table_%26_Backdate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authenticate the user to use BigQuery and Google Sheets.

Account must be a Google account with access to the projects required for the output and gmc tables.

In [ ]:
# Authenticate the user
from google.colab import auth
auth.authenticate_user()

# Import libraries needed
from google.cloud import bigquery

The below creates the partitioned table ready for populating with data.

Input required values in the form on the right.

In [ ]:
from google.cloud import bigquery

output_bq_project = "jellyfish-dcm-dt" #@param {type:"string"}
output_bq_dataset = "bissell" #@param {type:"string"}
output_clientname = "Bissell_DE" #@param {type:"string"}

client = bigquery.Client(project = output_bq_project)
# project = client.project
dataset_ref = bigquery.DatasetReference(output_bq_project, output_bq_dataset)

table_ref = dataset_ref.table("z_PriceCompetitivenessDashboard_" + output_clientname)
schema = [
  bigquery.SchemaField("date","DATE"),
  bigquery.SchemaField("Week","STRING"),
  bigquery.SchemaField("Year_Week","STRING"),
  bigquery.SchemaField("product_type","STRING"),
  bigquery.SchemaField("product_type_1","STRING"),
  bigquery.SchemaField("product_type_2","STRING"),
  bigquery.SchemaField("product_type_3","STRING"),
  bigquery.SchemaField("product_type_4","STRING"),
  bigquery.SchemaField("product_type_5","STRING"),
  bigquery.SchemaField("product_id","STRING"),
  bigquery.SchemaField("title","STRING"),
  bigquery.SchemaField("image_link","STRING"),
  bigquery.SchemaField("description","STRING"),
  bigquery.SchemaField("target_country","STRING"),
  bigquery.SchemaField("age_group","STRING"),
  bigquery.SchemaField("availability","STRING"),
  bigquery.SchemaField("brand","STRING"),
  bigquery.SchemaField("color","STRING"),
  bigquery.SchemaField("condition","STRING"),
  bigquery.SchemaField("gender","STRING"),
  bigquery.SchemaField("gtin","STRING"),
  bigquery.SchemaField("price_currency","STRING"),
  bigquery.SchemaField("google_product_category_path","STRING"),
  bigquery.SchemaField("google_category_1","STRING"),
  bigquery.SchemaField("google_category_2","STRING"),
  bigquery.SchemaField("google_category_3","STRING"),
  bigquery.SchemaField("google_category_4","STRING"),
  bigquery.SchemaField("google_category_5","STRING"),
  bigquery.SchemaField("custom_label_0","STRING"),
  bigquery.SchemaField("custom_label_1","STRING"),
  bigquery.SchemaField("custom_label_2","STRING"),
  bigquery.SchemaField("custom_label_3","STRING"),
  bigquery.SchemaField("custom_label_4","STRING"),
  bigquery.SchemaField("price_benchmark_country_of_sale","STRING"),
  bigquery.SchemaField("price_benchmark_currency","STRING"),
  bigquery.SchemaField("price","FLOAT"),
  bigquery.SchemaField("price_benchmark","FLOAT"),
  bigquery.SchemaField("price_benchmark_difference","FLOAT"),
  bigquery.SchemaField("product_data_from_date","DATE"),
  bigquery.SchemaField("benchmark_data_from_date","DATE")
]
table = bigquery.Table(table_ref, schema=schema)

table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,
    field="date",  # name of column to use for partitioning
    expiration_ms=None
)

table.require_partition_filter=True

table = client.create_table(table)

print(
    "Created table {}, partitioned on column {}".format(
        table.table_id, table.time_partitioning.field
    )
)

Created table z_PriceCompetitivenessDashboard_Bissell_DE, partitioned on column date


This query populates the newly created table with backdated data.

Input required values in the form on the right. 

In [ ]:
import time
from datetime import datetime
from datetime import date
from datetime import timedelta

gmc_data_bq_project = "jellyfish-dcm-dt" #@param {type:"string"}
gmc_data_bq_dataset = "bissell" #@param {type:"string"}
gmc_accountid =  143804479#@param {type:"integer"}

output_bq_project = "jellyfish-dcm-dt" #@param {type:"string"}
output_bq_dataset = "bissell" #@param {type:"string"}
output_clientname = "Bissell_DE" #@param {type:"string"}

first_partition_date = "2021-07-24" #@param {type:"date"}

benchmarks_table = "`" + gmc_data_bq_project + "." + gmc_data_bq_dataset + "." + "Products_PriceBenchmarks_" + str(gmc_accountid) + "`"
products_table = "`" + gmc_data_bq_project + "." + gmc_data_bq_dataset + "." + "Products_" + str(gmc_accountid) + "`"

client = bigquery.Client(project = output_bq_project)

run_date = datetime.strptime(first_partition_date, '%Y-%m-%d') #datetime object
run_date_str = run_date.strftime("%Y-%m-%d") #string
today_date_str = date.today().strftime("%Y-%m-%d") #string

while run_date_str != today_date_str:

  short_date = run_date.strftime("%Y%m%d")
  long_date = run_date.strftime("%Y-%m-%d")
  partition_date = (datetime.strptime(short_date, "%Y%m%d") - timedelta(days=1)).strftime("%Y%m%d")

  print("Executing " + long_date)

  job_config = bigquery.QueryJobConfig(destination = output_bq_project + "." + output_bq_dataset + "." + "z_PriceCompetitivenessDashboard_" + output_clientname + "$" + str(partition_date))
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
  
  try:
    
    query_job = client.query('''

WITH
  products AS (
  SELECT DISTINCT
    DATE(product_data_timestamp) AS date,
    product_id,
    title,
    product_type,
    image_link,
    description,
    target_country,
    age_group,
    availability,
    brand,
    color,
    condition,
    gender,
    gtin,

    CASE
      WHEN (sale_price.value IS NOT NULL AND sale_price_effective_start_date IS NULL AND sale_price_effective_end_date IS NULL)
            OR (price.value IS NOT NULL AND DATE(product_data_timestamp) >= DATE(sale_price_effective_start_date) AND DATE(product_data_timestamp) <= DATE(sale_price_effective_end_date))
            THEN sale_price.value
      ELSE price.value
      END AS price,
    
    price.currency AS price_currency,
    google_product_category_path,
    custom_labels.label_0 AS custom_label_0,
    custom_labels.label_1 AS custom_label_1,
    custom_labels.label_2 AS custom_label_2,
    custom_labels.label_3 AS custom_label_3,
    custom_labels.label_4 AS custom_label_4
  FROM
    {products_table}
  WHERE
    _PARTITIONDATE = "{long_date}"
  ),

  benchmarks AS (
  SELECT
    DATE(price_benchmark_timestamp) AS date,
    product_id,
    country_of_sale,
    price_benchmark_value AS price_benchmark,
    price_benchmark_currency
  FROM
    {benchmarks_table}
  WHERE
    _PARTITIONDATE = "{long_date}"
    )

SELECT DISTINCT
  DATE("{long_date}")-1 AS date,
  FORMAT_DATE("%V", pro.date) AS Week,
  FORMAT_DATE("%G-%V", pro.date) AS Year_Week,
  pro.product_type,
  SPLIT(pro.product_type," > ")[safe_OFFSET(0)] AS product_type_1,
  SPLIT(pro.product_type," > ")[safe_OFFSET(1)] AS product_type_2,
  SPLIT(pro.product_type," > ")[safe_OFFSET(2)] AS product_type_3,
  SPLIT(pro.product_type," > ")[safe_OFFSET(3)] AS product_type_4,
  SPLIT(pro.product_type," > ")[safe_OFFSET(4)] AS product_type_5,
  pro.product_id,
  pro.title,
  pro.image_link,
  pro.description,
  pro.target_country,
  pro.age_group,
  pro.availability,
  pro.brand,
  pro.color,
  pro.condition,
  pro.gender,
  pro.gtin,
  pro.price_currency,
  pro.google_product_category_path,
  SPLIT(pro.google_product_category_path," > ")[safe_OFFSET(0)] AS google_category_1,
  SPLIT(pro.google_product_category_path," > ")[safe_OFFSET(1)] AS google_category_2,
  SPLIT(pro.google_product_category_path," > ")[safe_OFFSET(2)] AS google_category_3,
  SPLIT(pro.google_product_category_path," > ")[safe_OFFSET(3)] AS google_category_4,
  SPLIT(pro.google_product_category_path," > ")[safe_OFFSET(4)] AS google_category_5,
  custom_label_0,
  custom_label_1,
  custom_label_2,
  custom_label_3,
  custom_label_4,
  ben.country_of_sale AS price_benchmark_country_of_sale,
  ben.price_benchmark_currency,
  CAST(pro.price AS FLOAT64) AS price,
  ben.price_benchmark,
  SAFE_DIVIDE(pro.price,ben.price_benchmark)-1 AS price_benchmark_difference,
  pro.date AS product_data_from_date,
  ben.date AS benchmark_data_from_date
    
FROM
  products pro

  LEFT JOIN
    benchmarks ben
  ON
    pro.product_id = ben.product_id
  AND
    pro.price_currency = ben.price_benchmark_currency

    '''.format(long_date = long_date, short_date = short_date, benchmarks_table = benchmarks_table, products_table = products_table), job_config=job_config)  

    df = query_job.result().to_dataframe()
    
  except:
    
    query_job.result()        
      
    print("Error querying table");

  run_date += timedelta(days=1)
  run_date_str = run_date.strftime("%Y-%m-%d")


Executing 2021-07-24
Executing 2021-07-25
Executing 2021-07-26
Executing 2021-07-27
Executing 2021-07-28
Executing 2021-07-29
Executing 2021-07-30
Executing 2021-07-31
Executing 2021-08-01
Executing 2021-08-02
Executing 2021-08-03
Executing 2021-08-04
Executing 2021-08-05
Executing 2021-08-06
Executing 2021-08-07
Executing 2021-08-08
Executing 2021-08-09
Executing 2021-08-10
Executing 2021-08-11
Executing 2021-08-12
Executing 2021-08-13
Executing 2021-08-14
Executing 2021-08-15
Executing 2021-08-16
Executing 2021-08-17
Executing 2021-08-18
Executing 2021-08-19
Executing 2021-08-20
Executing 2021-08-21
Executing 2021-08-22
Executing 2021-08-23
Executing 2021-08-24
Executing 2021-08-25
Executing 2021-08-26
Executing 2021-08-27
Executing 2021-08-28
Executing 2021-08-29
Executing 2021-08-30
Executing 2021-08-31
Executing 2021-09-01
Executing 2021-09-02
Executing 2021-09-03
Executing 2021-09-04
Executing 2021-09-05
Executing 2021-09-06
Executing 2021-09-07
Executing 2021-09-08
Executing 202